# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'12-01-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'12-01-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-12-02 05:27:41,33.93911,67.709953,46717,1797,36907,8013.0,Afghanistan,120.007683,3.846565
1,NaN,NaN,NaN,Albania,2020-12-02 05:27:41,41.15330,20.168300,39014,822,19384,18808.0,Albania,1355.688373,2.106936
2,NaN,NaN,NaN,Algeria,2020-12-02 05:27:41,28.03390,1.659600,84152,2447,54405,27300.0,Algeria,191.904215,2.907833
3,NaN,NaN,NaN,Andorra,2020-12-02 05:27:41,42.50630,1.521800,6790,76,5940,774.0,Andorra,8787.937617,1.119293
4,NaN,NaN,NaN,Angola,2020-12-02 05:27:41,-11.20270,17.873900,15251,350,7932,6969.0,Angola,46.403200,2.294931


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,44706,44988,45280,45490,45716,45839,45966,46215,46498,46717
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,32761,33556,34300,34944,35600,36245,36790,37625,38182,39014
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,74862,75867,77000,78025,79110,80168,81212,82221,83199,84152


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1687,1695,1712,1725,1737,1740,1752,1763,1774,1797
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,699,716,735,743,753,771,787,798,810,822
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2272,2294,2309,2329,2352,2372,2393,2410,2431,2447


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,35934,35976,36122,36145,36232,36295,36709,36731,36788,36907
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,15842,16230,16666,17031,17352,17755,18152,18481,18849,19384
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,48794,49421,50070,50712,51334,51946,52568,53204,53809,54405


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2617,2634,2661,2686,2704,2716,2735,2751,2780,2818
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,8199,8269,8376,8473,8576,8603,8733,8820,8890,9051
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1160,1161,1167,1170,1170,1171,1173,1175,1178,1189


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,39,39,39,41,42,42,42,42,42,42
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,84,84,84,98,98,98,98,98,98,137
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,10,10,10,10,10,10,10,11,11,11


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
648,1001.0,Autauga,Alabama,US,2020-12-02 05:27:41,32.539527,-86.644082,2818,42,0,2776.0,"Autauga, Alabama, US",5043.942079,1.490419
685,1075.0,Lamar,Alabama,US,2020-12-02 05:27:41,33.779950,-88.096680,701,8,0,693.0,"Lamar, Alabama, US",5077.870337,1.141227
686,1077.0,Lauderdale,Alabama,US,2020-12-02 05:27:41,34.901719,-87.656247,4267,54,0,4213.0,"Lauderdale, Alabama, US",4601.580951,1.265526


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,13721304,270642,5226581
India,9462809,137621,8889585
Brazil,6386787,173817,5712305


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,13721304,270642,5226581,8224081,2020-12-02 05:27:41,37.936303,-91.379001
India,9462809,137621,8889585,435603,2020-12-02 05:27:41,23.088275,81.806127
Brazil,6386787,173817,5712305,500665,2020-12-02 05:27:41,-12.669522,-48.480493
Russia,2302062,40050,1787962,474050,2020-12-02 05:27:41,54.546312,62.120860
France,2275429,52821,167915,2054693,2020-12-02 05:27:41,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
Texas,1250934,22114,0
California,1246042,19304,0
Florida,1008166,18679,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
Texas,1250934,22114,0,1228820,2020-12-02 05:27:41,31.660643,-98.653069
California,1246042,19304,0,1226738,2020-12-02 05:27:41,37.843962,-120.728594
Florida,1008166,18679,0,989487,2020-12-02 05:27:41,28.940755,-82.700744
Illinois,738846,13131,0,725715,2020-12-02 05:27:41,39.843618,-89.178330
New York,655265,34662,0,620603,2020-12-02 05:27:41,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,408396,7700,0
Illinois,Cook,310722,6695,0
Florida,Miami-Dade,231761,3839,0
Arizona,Maricopa,213121,4010,0
Texas,Harris,192127,3013,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,408396,7700,0,400696,2020-12-02 05:27:41,34.308284,-118.228241,6037.0
Illinois,Cook,310722,6695,0,304027,2020-12-02 05:27:41,41.841448,-87.816588,17031.0
Florida,Miami-Dade,231761,3839,0,227922,2020-12-02 05:27:41,25.611236,-80.551706,12086.0
Arizona,Maricopa,213121,4010,0,209111,2020-12-02 05:27:41,33.348359,-112.491815,4013.0
Texas,Harris,192127,3013,0,189114,2020-12-02 05:27:41,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-29,46215,37625,82221,6712,15103,141,1418807,134768,27902,279708,...,5716,72870,1,102040,1343,83585,10,2177,17608,9822
2020-11-30,46498,38182,83199,6745,15139,141,1424533,135124,27912,282456,...,5857,73094,1,102394,1347,85647,10,2191,17647,9950
2020-12-01,46717,39014,84152,6790,15251,142,1432570,135967,27923,285489,...,6024,73276,1,102621,1351,88004,10,2197,17665,10129


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-29,1763,798,2410,76,346,4,38473,2142,908,3105,...,76,608,0,894,35,717,1,617,357,275
2020-11-30,1774,810,2431,76,348,4,38730,2164,908,3184,...,77,610,0,897,35,732,1,619,357,276
2020-12-01,1797,822,2447,76,350,4,38928,2193,908,3325,...,78,611,0,901,35,747,1,619,357,277


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-29,36731,18481,53204,5794,7763,130,1249843,107364,25592,216998,...,4267,70151,0,96652,1179,63834,8,1504,16983,8472
2020-11-30,36788,18849,53809,5873,7851,130,1257227,108442,25602,221692,...,4357,70337,0,97326,1179,64987,8,1510,16998,8482
2020-12-01,36907,19384,54405,5940,7932,130,1263251,110365,25606,227497,...,4483,70534,0,97562,1195,65938,8,1514,17059,8643


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,2617,2634,2661,2686,2704,2716,2735,2751,2780,2818
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,622,626,633,650,667,671,679,683,689,701
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,3783,3829,3872,3936,3967,4004,4078,4133,4223,4267
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1214,1224,1239,1248,1263,1273,1279,1296,1316,1334
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,7504,7539,7597,7644,7718,7735,7757,7785,7814,7872


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-29,247229,32065,0,325995,156247,1215455,228772,112581,35251,49,...,366518,1225118,193809,4100,1544,235942,162700,46997,409054,32489
2020-11-30,249524,32576,0,326817,157359,1230264,232905,117295,35654,49,...,374493,1238752,195706,4172,1544,237835,165019,47842,411730,33305
2020-12-01,252900,32561,0,337139,159309,1246042,237310,118754,36343,49,...,380186,1250934,198216,4239,1550,240063,167216,48818,416365,33805


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-11-29        2751    8820    1175  1188   2946     700   1173    6366   
2020-11-30        2780    8890    1178  1196   2997     702   1178    6430   
2020-12-01        2818    9051    1189  1204   3061     701   1186    6598   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-29         1772     1029  ...    1287    330     1933      438   
2020-11-30         1779     1034  ...    1303    331     2000      430   
2020-12-01         1827     1051  ...    1347    337     2026      447   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-29           1812  1606  1046          0      428    396  
2020-11-30           1888  1644  1070          0      441    397  
2020-12-01           1903  1655  1089          0      468    409  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-29,3577,121,0,6634,2470,19151,3003,4961,770,0,...,4554,21843,868,67,23,4058,2703,729,3487,215
2020-11-30,3578,121,0,6639,2502,19173,3037,5020,772,0,...,4602,21896,871,69,23,4062,2774,735,3494,215
2020-12-01,3638,122,0,6687,2512,19304,3109,5040,777,0,...,4638,22114,890,72,23,4093,2805,758,3611,239


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-11-29          42      98      11   17     40      19     42     121   
2020-11-30          42      98      11   17     40      19     42     121   
2020-12-01          42     137      11   18     40      19     42     121   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-11-29           50       24  ...       6      7       10        1   
2020-11-30           50       24  ...       6      7       10        1   
2020-12-01           50       24  ...       6      8       10        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-29              6     2     4          0        8      1  
2020-11-30              6     2     4          0        8      1  
2020-12-01              6     2     4          9        8      2  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-29,0.005417,0.022696,0.012424,0.015431,0.001061,0.000000,0.003843,0.008788,0.000359,0.014681,...,0.037198,0.002090,0.0,0.002752,0.001491,0.020698,0.0,0.007870,0.001080,0.000000
2020-11-30,0.006124,0.014804,0.011895,0.004917,0.002384,0.000000,0.004036,0.002642,0.000358,0.009825,...,0.024668,0.003074,0.0,0.003469,0.002978,0.024669,0.0,0.006431,0.002215,0.013032
2020-12-01,0.004710,0.021790,0.011454,0.006672,0.007398,0.007092,0.005642,0.006239,0.000394,0.010738,...,0.028513,0.002490,0.0,0.002217,0.002970,0.027520,0.0,0.002738,0.001020,0.017990


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-29,0.006279,0.013977,0.007104,0.0,0.002899,0.0,0.003940,0.009901,0.001103,0.028827,...,0.013333,0.000000,NaN,0.002242,0.0,0.018466,0.0,0.003252,0.0,0.000000
2020-11-30,0.006239,0.015038,0.008714,0.0,0.005780,0.0,0.006680,0.010271,0.000000,0.025443,...,0.013158,0.003289,NaN,0.003356,0.0,0.020921,0.0,0.003241,0.0,0.003636
2020-12-01,0.012965,0.014815,0.006582,0.0,0.005747,0.0,0.005112,0.013401,0.000000,0.044284,...,0.012987,0.001639,NaN,0.004459,0.0,0.020492,0.0,0.000000,0.0,0.003623


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-29,0.000599,0.018125,0.012099,0.014711,0.000000,0.0,0.005605,0.008311,0.000156,0.029906,...,0.019594,0.002444,NaN,0.000000,0.000000,0.014140,0.0,0.004005,0.003427,0.000000
2020-11-30,0.001552,0.019912,0.011371,0.013635,0.011336,0.0,0.005908,0.010041,0.000391,0.021632,...,0.021092,0.002651,NaN,0.006973,0.000000,0.018062,0.0,0.003989,0.000883,0.001180
2020-12-01,0.003235,0.028383,0.011076,0.011408,0.010317,0.0,0.004791,0.017733,0.000156,0.026185,...,0.028919,0.002801,NaN,0.002425,0.013571,0.014634,0.0,0.002649,0.003589,0.018981


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-29,0.009127,0.019749,NaN,0.009979,0.007876,0.012353,0.015487,0.000000,0.016758,0.0,...,0.008397,0.009510,0.008965,0.016613,0.003901,0.009952,0.012862,0.025128,0.010012,0.017571
2020-11-30,0.009283,0.015936,NaN,0.002522,0.007117,0.012184,0.018066,0.041872,0.011432,0.0,...,0.021759,0.011129,0.009788,0.017561,0.000000,0.008023,0.014253,0.017980,0.006542,0.025116
2020-12-01,0.013530,-0.000460,NaN,0.031583,0.012392,0.012825,0.018913,0.012439,0.019325,0.0,...,0.015202,0.009834,0.012825,0.016059,0.003886,0.009368,0.013314,0.020400,0.011257,0.015013


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-29      0.005850  0.009962  0.001705  0.007634  0.008214  0.005747   
2020-11-30      0.010542  0.007937  0.002553  0.006734  0.017312  0.002857   
2020-12-01      0.013669  0.018110  0.009338  0.006689  0.021355 -0.001425   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-29      0.006867  0.010316  0.002262  0.006849  ...  0.010998   
2020-11-30      0.004263  0.010053  0.003950  0.004859  ...  0.012432   
2020-12-01      0.006791  0.026128  0.026981  0.016441  ...  0.033768   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-29      0.021672  0.019515  0.020979   0.017978  0.022930  0.008679   
2020-11-30      0.003030  0.034661 -0.018265   0.041943  0.023661  0.022945   
2020-12-01      0.018127  0.013000  0.039535   0.007945  0.006691  0.017757   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-29            NaN  0.026379  0.000000  
2020-11-30            NaN  0.030374  0.002525  
2020-12-01            NaN  0.061224  0.030227  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-29,0.001400,0.000000,NaN,0.001510,0.008575,0.001831,0.006705,0.000000,0.009174,NaN,...,0.002863,0.002524,0.005794,0.000000,0.0,0.000987,0.000000,0.015320,0.006640,0.000000
2020-11-30,0.000280,0.000000,NaN,0.000754,0.012955,0.001149,0.011322,0.011893,0.002597,NaN,...,0.010540,0.002426,0.003456,0.029851,0.0,0.000986,0.026267,0.008230,0.002007,0.000000
2020-12-01,0.016769,0.008264,NaN,0.007230,0.003997,0.006833,0.023708,0.003984,0.006477,NaN,...,0.007823,0.009956,0.021814,0.043478,0.0,0.007632,0.011175,0.031293,0.033486,0.111628


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga   Baldwin Barbour      Bibb Blount Bullock   Butler   
2020-11-29         0.0  0.000000     0.1  0.000000    0.0     0.0  0.02439   
2020-11-30         0.0  0.000000     0.0  0.000000    0.0     0.0  0.00000   
2020-12-01         0.0  0.397959     0.0  0.058824    0.0     0.0  0.00000   

Province_State                              ... Wyoming                     \
Admin2           Calhoun Chambers Cherokee  ...    Park    Platte Sheridan   
2020-11-29      0.016807      0.0      0.0  ...     0.0  0.000000      0.0   
2020-11-30      0.000000      0.0      0.0  ...     0.0  0.000000      0.0   
2020-12-01      0.000000      0.0      0.0  ...     0.0  0.142857      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-11-29          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2020-11-30          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2020-12-01          0.0        0.0   0.0   0.0        inf      0.0    1.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-29,0.004375,0.019833,0.012877,0.010828,0.003423,0.000681,0.004449,0.009626,0.000335,0.016584,...,0.035975,0.002279,0.0,0.002795,0.002550,0.020842,3.801265e-50,0.006604,0.001164,0.005354
2020-11-30,0.005249,0.017319,0.012386,0.007872,0.002903,0.000340,0.004242,0.006134,0.000347,0.013204,...,0.030321,0.002677,0.0,0.003132,0.002764,0.022756,1.900633e-50,0.006518,0.001689,0.009193
2020-12-01,0.004980,0.019555,0.011920,0.007272,0.005151,0.003716,0.004942,0.006186,0.000370,0.011971,...,0.029417,0.002583,0.0,0.002674,0.002867,0.025138,9.503163e-51,0.004628,0.001355,0.013592


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-29,0.006000,0.017105,0.007953,4.069187e-07,0.004457,0.000005,0.004367,0.011847,0.000551,0.035778,...,0.011680,0.000636,NaN,0.003334,1.285047e-28,0.018719,0.0,0.002661,0.000013,0.000713
2020-11-30,0.006120,0.016071,0.008334,2.034593e-07,0.005119,0.000003,0.005523,0.011059,0.000276,0.030610,...,0.012419,0.001963,NaN,0.003345,6.425235e-29,0.019820,0.0,0.002951,0.000007,0.002175
2020-12-01,0.009542,0.015443,0.007458,1.017297e-07,0.005433,0.000001,0.005318,0.012230,0.000138,0.037447,...,0.012703,0.001801,NaN,0.003902,3.212618e-29,0.020156,0.0,0.001476,0.000003,0.002899


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-11-29,0.003673,0.020235,0.012107,0.010593,0.004924,0.000744,0.006077,0.012708,0.000254,0.033266,...,0.020147,0.002506,NaN,0.001572,0.003154,0.013801,2.227304e-52,0.004523,0.003599,0.001818
2020-11-30,0.002613,0.020074,0.011739,0.012114,0.008130,0.000372,0.005992,0.011374,0.000322,0.027449,...,0.020620,0.002579,NaN,0.004273,0.001577,0.015932,1.113652e-52,0.004256,0.002241,0.001499
2020-12-01,0.002924,0.024229,0.011408,0.011761,0.009223,0.000186,0.005392,0.014554,0.000239,0.026817,...,0.024769,0.002690,NaN,0.003349,0.007574,0.015283,5.568260e-53,0.003453,0.002915,0.010240


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-29,0.008541,0.020653,NaN,0.011467,0.008757,0.012247,0.018205,0.005000,0.016104,7.349931e-79,...,0.011701,0.007641,0.012143,0.017013,0.004966,0.010737,0.015157,0.022498,0.010735,0.015784
2020-11-30,0.008912,0.018295,NaN,0.006994,0.007937,0.012215,0.018136,0.023436,0.013768,3.674965e-79,...,0.016730,0.009385,0.010966,0.017287,0.002483,0.009380,0.014705,0.020239,0.008638,0.020450
2020-12-01,0.011221,0.008917,NaN,0.019289,0.010165,0.012520,0.018524,0.017937,0.016546,1.837483e-79,...,0.015966,0.009610,0.011896,0.016673,0.003184,0.009374,0.014009,0.020320,0.009948,0.017732


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-11-29      0.006214  0.010621  0.001584  0.006645  0.008762  0.003947   
2020-11-30      0.008378  0.009279  0.002069  0.006690  0.013037  0.003402   
2020-12-01      0.011024  0.013694  0.005703  0.006689  0.017196  0.000989   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-11-29      0.006702  0.009721  0.003404  0.006495  ...  0.010454   
2020-11-30      0.005482  0.009887  0.003677  0.005677  ...  0.011443   
2020-12-01      0.006137  0.018007  0.015329  0.011059  ...  0.022606   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-11-29      0.019468  0.017462  0.018391   0.025416  0.022501  0.008954   
2020-11-30      0.011249  0.026062  0.000063   0.033680  0.023081  0.015949   
2020-12-01      0.014688  0.019531  0.019799   0.020812  0.014886  0.016853   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-11-29           -1.0  0.027189  0.004056  
2020-11-30           -1.0  0.028781  0.003291  
2020-12-01           -1.0  0.045003  0.016759  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-11-29,0.002040,0.007856,NaN,0.003113,0.006450,0.002179,0.006673,0.001286,0.005285,NaN,...,0.004278,0.003727,0.010631,0.006167,1.032116e-09,0.002311,0.000442,0.012427,0.008900,0.003243
2020-11-30,0.001160,0.003928,NaN,0.001934,0.009703,0.001664,0.008998,0.006589,0.003941,NaN,...,0.007409,0.003077,0.007044,0.018009,5.160579e-10,0.001648,0.013354,0.010329,0.005454,0.001621
2020-12-01,0.008964,0.006096,NaN,0.004582,0.006850,0.004248,0.016353,0.005287,0.005209,NaN,...,0.007616,0.006516,0.014429,0.030744,2.580289e-10,0.004640,0.012265,0.020811,0.019470,0.056625


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2020-11-29      0.003158  0.005208  0.050027 -0.000094  0.004270   
2020-11-30      0.001579  0.002604  0.025014 -0.000047  0.002135   
2020-12-01      0.000789  0.200282  0.012507  0.029388  0.001068   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2020-11-29      1.620623e-07  0.012195  0.011288  0.001927  0.000260  ...   
2020-11-30      8.103115e-08  0.006098  0.005644  0.000963  0.000130  ...   
2020-12-01      4.051558e-08  0.003049  0.002822  0.000482  0.000065  ...   

Province_State   Wyoming                                          \
Admin2              Park    Platte  Sheridan Sublette Sweetwater   
2020-11-29      0.003911  0.006775  0.000070      0.0   0.000244   
2020-11-30      0.001956  0.003387  0.000035      0.0   0.000122   
2020-12-01      0.000978  0.073122  0.000017      0.0   0.000061   

Province_State                                                                 
Admin2                 Teton         Uinta Unassigned  Washakie        Weston  
2020-11-29      7.450581e-09  7.947468e-08  -0.972376  0.004464 -2.422029e-07  
2020-11-30      3.725290e-09  3.973734e-08  -0.972376  0.002232 -1.201479e-07  
2020-12-01      1.862645e-09  1.986867e-08  -0.972376  0.001116  5.019607e-01  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201201,AK,31834,NaN,986794,NaN,totalTestsViral,1018628,155.0,749.0,...,1018628,1,24,6c0d9dbaedf0d16c7d345f9928e37ce2dba3a0ec,0,0,0,0,0,NaN
1,20201201,AL,252900,42863.0,1383805,NaN,totalTestsViral,1593842,1785.0,25610.0,...,1636705,60,272,ff146d1c2df88ca7b0ac8e94dacc310424c5f0ba,0,0,0,0,0,NaN
2,20201201,AR,159309,20111.0,1552255,NaN,totalTestsViral,1691453,1074.0,9021.0,...,1711564,10,84,3b0277995180be5c38d5580749efebeba89c0629,0,0,0,0,0,NaN
3,20201201,AS,0,NaN,1988,NaN,totalTestsViral,1988,NaN,NaN,...,1988,0,0,6624db5a80c3ab56003683c32575976c74b534aa,0,0,0,0,0,NaN
4,20201201,AZ,337139,10301.0,1945076,NaN,totalTestsPeopleViral,2271914,2594.0,26072.0,...,2282215,48,286,0ecbb6723a2c67fe320a84513e6538154490caab,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201201,AK,31834.0,NaN,986794.0,NaN,totalTestsViral,1018628.0,155.0,749.0,...,1018628,1,24,6c0d9dbaedf0d16c7d345f9928e37ce2dba3a0ec,0,0,0,0,0,NaN
1,20201201,AL,252900.0,42863.0,1383805.0,NaN,totalTestsViral,1593842.0,1785.0,25610.0,...,1636705,60,272,ff146d1c2df88ca7b0ac8e94dacc310424c5f0ba,0,0,0,0,0,NaN
2,20201201,AR,159309.0,20111.0,1552255.0,NaN,totalTestsViral,1691453.0,1074.0,9021.0,...,1711564,10,84,3b0277995180be5c38d5580749efebeba89c0629,0,0,0,0,0,NaN
3,20201201,AS,0.0,NaN,1988.0,NaN,totalTestsViral,1988.0,NaN,NaN,...,1988,0,0,6624db5a80c3ab56003683c32575976c74b534aa,0,0,0,0,0,NaN
4,20201201,AZ,337139.0,10301.0,1945076.0,NaN,totalTestsPeopleViral,2271914.0,2594.0,26072.0,...,2282215,48,286,0ecbb6723a2c67fe320a84513e6538154490caab,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-01,AK,31834,NaN,986794,NaN,totalTestsViral,1018628,155.0,749.0,NaN,...,1018628,1,24,6c0d9dbaedf0d16c7d345f9928e37ce2dba3a0ec,0,0,0,0,0,NaN
2020-12-01,AL,252900,42863.0,1383805,NaN,totalTestsViral,1593842,1785.0,25610.0,NaN,...,1636705,60,272,ff146d1c2df88ca7b0ac8e94dacc310424c5f0ba,0,0,0,0,0,NaN
2020-12-01,AR,159309,20111.0,1552255,NaN,totalTestsViral,1691453,1074.0,9021.0,403.0,...,1711564,10,84,3b0277995180be5c38d5580749efebeba89c0629,0,0,0,0,0,NaN
2020-12-01,AS,0,NaN,1988,NaN,totalTestsViral,1988,NaN,NaN,NaN,...,1988,0,0,6624db5a80c3ab56003683c32575976c74b534aa,0,0,0,0,0,NaN
2020-12-01,AZ,337139,10301.0,1945076,NaN,totalTestsPeopleViral,2271914,2594.0,26072.0,597.0,...,2282215,48,286,0ecbb6723a2c67fe320a84513e6538154490caab,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-01,AK,31834.0,NaN,986794.0,NaN,totalTestsViral,1018628.0,155.0,749.0,NaN,...,1018628,1,24,6c0d9dbaedf0d16c7d345f9928e37ce2dba3a0ec,0,0,0,0,0,NaN
2020-12-01,AL,252900.0,42863.0,1383805.0,NaN,totalTestsViral,1593842.0,1785.0,25610.0,NaN,...,1636705,60,272,ff146d1c2df88ca7b0ac8e94dacc310424c5f0ba,0,0,0,0,0,NaN
2020-12-01,AR,159309.0,20111.0,1552255.0,NaN,totalTestsViral,1691453.0,1074.0,9021.0,403.0,...,1711564,10,84,3b0277995180be5c38d5580749efebeba89c0629,0,0,0,0,0,NaN
2020-12-01,AS,0.0,NaN,1988.0,NaN,totalTestsViral,1988.0,NaN,NaN,NaN,...,1988,0,0,6624db5a80c3ab56003683c32575976c74b534aa,0,0,0,0,0,NaN
2020-12-01,AZ,337139.0,10301.0,1945076.0,NaN,totalTestsPeopleViral,2271914.0,2594.0,26072.0,597.0,...,2282215,48,286,0ecbb6723a2c67fe320a84513e6538154490caab,0,0,0,0,0,NaN
